In [79]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults

In [80]:
tool = TavilySearchResults(tavily_api_key = "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", max_results=2)

In [81]:
# Create our agent state
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

## Persistence
We are going to add in persistence. In order to deal with persistence, we've added the concept of a __check pointer__ into LangGraph.
A __check pointer__ basically checkpoints, the state,  after and between every node.
To add persistence for this agent, we use SqliteSaver, it is really simple checkpointer that we've added that uses Sqlite a built in 
database under the hood.

In [82]:
from langgraph.checkpoint.sqlite import SqliteSaver
# memory = SqliteSaver.from_conn_string(":memory:")

from contextlib import ExitStack

# Use contextlib to manually enter the context manager and keep the object
stack = ExitStack()
# Create the memory context manager and enter the context
memory= stack.enter_context(SqliteSaver.from_conn_string(":memory:"))

In [83]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [100]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOllama(model = 'qwen2.5', streaming = True)
abot = Agent(model,
             [tool],
             system=prompt,
             checkpointer=memory)
                   

In [101]:
messages = [HumanMessage(content="What is the weather in sf?")]

In [86]:
thread = {"configurable": {"thread_id": "1"}}

In [87]:
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v['messages'])

[AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5', 'created_at': '2025-06-17T08:45:23.599223444Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4029793473, 'load_duration': 11802515, 'prompt_eval_count': 243, 'prompt_eval_duration': 246166287, 'eval_count': 28, 'eval_duration': 3768376238, 'model_name': 'qwen2.5'}, id='run--36427003-568e-41d3-81ec-746b367b64e6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in san francisco'}, 'id': '2e627026-6eb7-4724-8f95-a45b03d1a8ee', 'type': 'tool_call'}], usage_metadata={'input_tokens': 243, 'output_tokens': 28, 'total_tokens': 271})]
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in san francisco'}, 'id': '2e627026-6eb7-4724-8f95-a45b03d1a8ee', 'type': 'tool_call'}
Back to the model!
[ToolMessage(content='[{\'title\': \'Tuesday, June 17, 2025. San Francisco, CA - Weather Forecast\', \'url\': \'https://weathershogun.com/weather/usa/ca/san-fran

In [88]:
messages = [HumanMessage(content="What about in Lahore Pakistan")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5', 'created_at': '2025-06-17T08:48:00.263886854Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4203560170, 'load_duration': 12580680, 'prompt_eval_count': 1305, 'prompt_eval_duration': 355458706, 'eval_count': 27, 'eval_duration': 3807003689, 'model_name': 'qwen2.5'}, id='run--100f9f8b-8a66-4dec-8fbb-72adb2c21424-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in lahore pakistan'}, 'id': 'b39153c1-9c1b-42f1-a50a-d6ec86920eaf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1305, 'output_tokens': 27, 'total_tokens': 1332})]}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in lahore pakistan'}, 'id': 'b39153c1-9c1b-42f1-a50a-d6ec86920eaf', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'title\': \'Weather in Lahore in June 2025 - Detailed Forecast\', \'url\': \'https://www.easeweather.c

In [89]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='Based on the forecasts, June 17, 2025, in Lahore will be partly cloudy with a high of 47°C and a low of 35°C. On June 18, 2025, it is expected to be sunny with similar temperatures, ranging from 47°C during the day to 36°C at night.\n\nSince both days have the same maximum temperature of 47°C, neither day will be warmer than the other based on the provided information. They are expected to have comparable temperatures.', additional_kwargs={}, response_metadata={'model': 'qwen2.5', 'created_at': '2025-06-17T08:50:41.763877392Z', 'done': True, 'done_reason': 'stop', 'total_duration': 20419323592, 'load_duration': 19228296, 'prompt_eval_count': 2496, 'prompt_eval_duration': 395930880, 'eval_count': 115, 'eval_duration': 19936905164, 'model_name': 'qwen2.5'}, id='run--56e824ed-e0cc-4728-ac5e-6d3f6986689e-0', usage_metadata={'input_tokens': 2496, 'output_tokens': 115, 'total_tokens': 2611})]}


## Let's change thread id

In [90]:
messages = [HumanMessage(content="Which one is warmer?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='I need more information to determine which one is warmer. Could you please specify what two things or objects you are comparing?', additional_kwargs={}, response_metadata={'model': 'qwen2.5', 'created_at': '2025-06-17T08:52:09.113102903Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3734056093, 'load_duration': 12634039, 'prompt_eval_count': 241, 'prompt_eval_duration': 262737225, 'eval_count': 25, 'eval_duration': 3455083324, 'model_name': 'qwen2.5'}, id='run--cc4b071e-fbfc-4587-b5ec-030a5b9e5d25-0', usage_metadata={'input_tokens': 241, 'output_tokens': 25, 'total_tokens': 266})]}


In [91]:
#close connection
stack.close()

# Let's stream the tokkens now

In [102]:
import asyncio
from contextlib import AsyncExitStack
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
stack = AsyncExitStack()
memory = await stack.enter_async_context(AsyncSqliteSaver.from_conn_string(":memory:"))

abot = Agent(model, [tool], system=prompt, checkpointer=memory)

messages = [HumanMessage(content="What is the weather in SF?")]
thread = {"configurable": {"thread_id": "4"}}
async for event in abot.graph.astream_events({"messages": messages}, thread, version="v1"):
        kind = event["event"]
        if kind == "on_chat_model_stream":
            content = event["data"]["chunk"].content
            if content:
                # Empty content in the context of OpenAI means
                # that the model is asking for a tool to be invoked.
                # So we only print non-empty content
                print(content, end="|")

await stack.aclose()

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': '8bf13700-7615-428f-81ca-8c9096d58a06', 'type': 'tool_call'}
Back to the model!
The current weather forecast for San Francisco, California on Tuesday, June 17, 2025 is as follows:

- Daytime temperature: 70°F
- Nighttime temperature: 55°F
- Precipitation: 0%
- Wind speed: 12 mph
- UV Index: 11

The forecast suggests clear skies with minimal chance of precipitation. Make sure to check the latest updates closer to your date as conditions can change.|